In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from deepmultilingualpunctuation import PunctuationModel
from time import sleep
from bs4 import BeautifulSoup
from html2text import html2text
from datetime import date,timedelta
import pandas as pd
import locale
import re

# locale.setlocale(locale.LC_TIME, 'es_ES')
# video_url = 'https://www.youtube.com/playlist?list=PL_WZQpo8BKJAcULurjmlCyqgl-sT-DMlG'
video_url = 'https://www.youtube.com/playlist?list=PLExodH-pz2mqYsQn1rxwGEqQjhVX8B8r6'
# video_url = 'https://www.youtube.com/playlist?list=PLzuFY9Ixj9Z4G5-eRHblrmwMOY7tLUCHi'
model_punctuation = PunctuationModel(model = "kredor/punctuate-all")


options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("detach", True)
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get(video_url)
driver.maximize_window()
sleep(2)
j=0
while j<1:
    driver.execute_script("window.scrollBy(0, 999999);")
    sleep(2 + j)
    j+=1
driver.execute_script("window.scrollBy(0, -999999);")
soup = BeautifulSoup(driver.page_source, 'html.parser')
list_videos = soup.find_all("ytd-playlist-video-renderer", {'class':'style-scope ytd-playlist-video-list-renderer'})
url_vid = [('https://www.youtube.com' + i.find("a", {'class':'yt-simple-endpoint style-scope ytd-playlist-video-renderer'})['href'].split('&')[0])
             for i in list_videos]
title_vid = [(i.find("a", {'class':'yt-simple-endpoint style-scope ytd-playlist-video-renderer'})['title'])
                 for i in list_videos]
author_vid = [('https://www.youtube.com' + i.find("a", {'class':'yt-simple-endpoint style-scope yt-formatted-string'})['href'])
                 for i in list_videos]
transc_t_vid = []; transc_r_vid = []; icon_img_vid=[];desc_vid=[];views_vid=[];date_vid=[]
for j in url_vid[:2]:
    driver.get(j)
    sleep(2)
    driver.find_element(By.XPATH, '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div/div/div/div/ytd-watch-metadata/div/div/div/div/div/ytd-menu-renderer/yt-button-shape/button/yt-touch-feedback-shape/div[@class=\"yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response"]').click()
    driver.find_element(By.XPATH, '/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer[@class=\"style-scope ytd-menu-popup-renderer"]').click()
    sleep(4)
    driver.find_element(By.XPATH, '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div/div/div/div/ytd-watch-metadata/div/div/div/div/ytd-text-inline-expander/tp-yt-paper-button[@id=\"expand"]').click()
    sleep(4)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_transcripsion = soup.find_all("div", {'class':re.compile(f'style-scope ytd-transcript-search-panel-renderer')})
    time_transc = [item.get_text(strip=True) for item in list_transcripsion[1].find_all(class_='segment-timestamp style-scope ytd-transcript-segment-renderer')]
    text_transc = [re.sub(patron, '', item.get_text(strip=True)) for item in list_transcripsion[1].find_all(class_='segment-text style-scope ytd-transcript-segment-renderer')]
    transc_vid = " ".join(text_transc)
    transc_t_vid.append("||".join([time_transc[i]+" - "+text_transc[i] for i in range(len(time_transc))]))
    transc_r= model_punctuation.restore_punctuation(transc_vid)
    transc_r_vid.append(''.join([transc_r[g].upper() if (g in ([g+2 for g in range(len(transc_r)) if transc_r[g]=="."][:-1]) or g==0)
     else transc_r[g] for g in range(len(transc_r))]))
    icon_img_vid.append(soup.find("link", {'as':'image'})['href'])
    desc_vid.append((soup.find_all("span", {'class':re.compile(f'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap')}))[0].get_text(strip=True))
    views_vid.append(int((soup.find_all("span", {'class':re.compile(f'style-scope yt-formatted-string bold')}))[0].get_text(strip=True).split(" ")[0].replace(",","")))
    date_vid.append(("/".join((soup.find_all("span", {'class':re.compile(f'style-scope yt-formatted-string bold')}))[2].get_text(strip=True).split(" ")[-3:])).upper())

C:\Users\vladi\anaconda3\envs\NLP_curso\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\vladi\anaconda3\envs\NLP_curso\lib\site-packages\transformers\pipelines\token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(
[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:01<00:00, 3.31MB/s]


NameError: name 'patron' is not defined

In [157]:
df_list_videos= pd.DataFrame({
        'title_vid': title_vid,
        'url_vid': url_vid,
        'author_vid': author_vid,
        'icon_img_vid': icon_img_vid,
        'transc_t_vid': transc_t_vid,
        'transc_r_vid': transc_r_vid,
        'desc_vid': desc_vid,
        'views_vid': views_vid,
        'date_vid': date_vid})

In [158]:
df_list_videos

,title_vid,url_vid,author_vid,icon_img_vid,transc_t_vid,transc_r_vid,desc_vid,views_vid,date_vid
0,🇬🇹 Que ver en ANTIGUA la joya colonial de Guat...,https://www.youtube.com/watch?v=wRw1cbDxZ_8,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/wRw1cbDxZ_8/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿Vas a viajar a ANTIGUA? Haz click aquí \n👇 ...,125530,5/FEB/2017
1,"🇨🇷 Turismo en COSTA RICA. Animales, playas y n...",https://www.youtube.com/watch?v=0DWmGz4_znk,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/0DWmGz4_znk/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VIAJAR A COSTA RICA? El es país con los...,66417,23/OCT/2016
2,🇵🇦 Qué ver en PANAMA. Lo mejor del país del canal,https://www.youtube.com/watch?v=udf6dG5SuUI,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/udf6dG5SuUI/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",👉📸👉¿Vas visitar a#PANAMÁ? Contrata desde este ...,1665427,31/OCT/2016
3,"🇸🇻 Qué ver en EL SALVADOR, el pulgarcito de Am...",https://www.youtube.com/watch?v=np15C5kuGlk,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/np15C5kuGlk/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿Es seguro viajar a EL SALVADOR? Infórmate aqu...,166213,31/JUL/2016
4,🇳🇮 Qué ver en GRANADA la ciudad más bonita de ...,https://www.youtube.com/watch?v=S8suxdW8c44,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/S8suxdW8c44/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VISITAR GRANADA? Posiblemente sea el pu...,224734,11/SEPT/2016
5,"🇨🇷 Experiencias en COSTA RICA: rafting, tirole...",https://www.youtube.com/watch?v=UKsACg4t8tU,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/UKsACg4t8tU/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VIAJAR A COSTA RICA? En este vídeo tien...,21309,18/DIC/2016
6,🇨🇷 Qué ver en SAN JOSÉ capital de Costa Rica,https://www.youtube.com/watch?v=721n0YBh9GQ,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/721n0YBh9GQ/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...","¿VAS A VIAJAS A COSTA RICA? Este país, además ...",287936,9/OCT/2016
7,🇳🇮 Qué ver en MANAGUA la capital de Nicaragua,https://www.youtube.com/watch?v=16ApxsR3BpY,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/16ApxsR3BpY/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿CONOCES NICARAGUA? Es uno de los países más b...,220467,28/AGO/2016
8,"🇳🇮 Qué ver en LEÓN, cuna de Rubén Darío en Nic...",https://www.youtube.com/watch?v=AsWalNVaRA8,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/AsWalNVaRA8/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VIAJAR A LEÓN? La ciudad de Rubén Darío...,101456,1/OCT/2016
9,"🇭🇳 ROATÁN, el Caribe mas bello del mundo en Ho...",https://www.youtube.com/watch?v=8qJyM6Mdnts,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/8qJyM6Mdnts/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola amigos, bienvenidos a una nueva aventura ...",🐠 ¿Vas a viajar a Roatán? En este artículo te ...,696398,25/SEPT/2016


In [159]:
df_list_videos.to_csv('df_list_videos.csv', sep=';', index = True)

In [8]:
import pandas as pd
df_list_videos=pd.read_csv("df_list_videos.csv", sep=';')
texto_vid=df_list_videos["transc_r_vid"]

In [9]:
texto_vid[1]

'Hola, amigos, bienvenidos a una nueva aventura hoy por los parques naturales de costa rica. Yo ya estoy preparado para descubrir la naturaleza de costa rica, vosotros también, pues una vida. Se puede decir sin miedo a equivocarnos que costa rica es toda ella, un gran parque natural. A pesar de ocupar tan sólo el 0, 03 por ciento de la superficie terrestre, atesora el 5 por ciento de la biodiversidad mundial y ella sola tiene más variedad de aves autóctonas que eeuu y canadá juntos. Vamos a recorrer algunos de sus más emblemáticos espacios naturales y empezamos por el parque nacional volcán arenal. Una de las actividades más populares de este parque son las caminatas por sus famosos puentes colgantes, todo un desafío para los que padecen de vértigo y mal de alturas. Nuestra siguiente visita nos lleva al bosque nuboso monte verde, una selva tropical de más de 10.000 hectáreas. Este bosque es uno de los destinos favoritos para los amantes de los avistamientos de pájaros, ya que a lo larg

In [164]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, pipeline

In [165]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [167]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
device = torch.device('cpu')

In [168]:
def extraer_fragmentos(texto):
    fragmentos = []
    punto_contador = 0
    fragmento_actual = ''
    ultimo_fragmento = False
    for caracter in texto:
        fragmento_actual += caracter
        if caracter == '.':
            punto_contador += 1
            if punto_contador == 5:
                fragmentos.append(fragmento_actual)
                fragmento_actual = ''
                punto_contador = 0
        if len(fragmentos) > 0 and punto_contador == 0:
            ultimo_fragmento = True
    # Si no se completaron los 5 puntos al final del texto, se agrega el último fragmento
    if ultimo_fragmento and fragmento_actual.strip() != '':
        fragmentos.append(fragmento_actual)
    return fragmentos

def generar_resumenes(valor):
    resumenes = []
    resumen_concatenado = ''
    fragmentos = extraer_fragmentos(valor)
    for fragmento in fragmentos:
        preprocessed_text = fragmento.strip().replace("\n", "")
        t5_prepared_text = "summarize: " + preprocessed_text
        #print("Texto original preprocesado:\n", preprocessed_text)

        # Tokenizar el texto
        tokenized_text = tokenizer.encode(t5_prepared_text, return_tensors="pt").to(device)

        # Generar el resumen
        summary_ids = model.generate(
            tokenized_text, num_beams=4,no_repeat_ngram_size=2,
            min_length=30,max_length=100,early_stopping=True)

        # Decodificar el resumen
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        resumenes.append(output)
        resumen_concatenado += output + ' '

        #print("\n\nResumen del texto:\n", output)

    return resumen_concatenado



In [189]:
summary_vid = [generar_resumenes(h) for h in list(texto_vid)]
df_list_videos["summary_vid"] = summary_vid

In [190]:
df_list_videos.to_csv('df_list_videos.csv', sep=';', index = True)

In [191]:
df_list_videos

,title_vid,url_vid,author_vid,icon_img_vid,transc_t_vid,transc_r_vid,desc_vid,views_vid,date_vid,summary_vid
0,🇬🇹 Que ver en ANTIGUA la joya colonial de Guat...,https://www.youtube.com/watch?v=wRw1cbDxZ_8,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/wRw1cbDxZ_8/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿Vas a viajar a ANTIGUA? Haz click aquí \n👇 ...,125530,5/FEB/2017,Vamos a comenzar nuestra visita por antigua en...
1,"🇨🇷 Turismo en COSTA RICA. Animales, playas y n...",https://www.youtube.com/watch?v=0DWmGz4_znk,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/0DWmGz4_znk/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VIAJAR A COSTA RICA? El es país con los...,66417,23/OCT/2016,Costa rica atesora el 5 por ciento of la biodi...
2,🇵🇦 Qué ver en PANAMA. Lo mejor del país del canal,https://www.youtube.com/watch?v=udf6dG5SuUI,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/udf6dG5SuUI/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",👉📸👉¿Vas visitar a#PANAMÁ? Contrata desde este ...,1665427,31/OCT/2016,La ciudad de panamá tiene muchas caras. La más...
3,"🇸🇻 Qué ver en EL SALVADOR, el pulgarcito de Am...",https://www.youtube.com/watch?v=np15C5kuGlk,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/np15C5kuGlk/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿Es seguro viajar a EL SALVADOR? Infórmate aqu...,166213,31/JUL/2016,"Hola, amigos, bienvenidos a una nueva aventura..."
4,🇳🇮 Qué ver en GRANADA la ciudad más bonita de ...,https://www.youtube.com/watch?v=S8suxdW8c44,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/S8suxdW8c44/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VISITAR GRANADA? Posiblemente sea el pu...,224734,11/SEPT/2016,Granada is una de las ciudades coloniales más ...
5,"🇨🇷 Experiencias en COSTA RICA: rafting, tirole...",https://www.youtube.com/watch?v=UKsACg4t8tU,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/UKsACg4t8tU/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VIAJAR A COSTA RICA? En este vídeo tien...,21309,18/DIC/2016,Hoy vamos a experimentar costa rica. Yo ya est...
6,🇨🇷 Qué ver en SAN JOSÉ capital de Costa Rica,https://www.youtube.com/watch?v=721n0YBh9GQ,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/721n0YBh9GQ/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...","¿VAS A VIAJAS A COSTA RICA? Este país, además ...",287936,9/OCT/2016,"Vídeo es de san josé, la capital de costa rica..."
7,🇳🇮 Qué ver en MANAGUA la capital de Nicaragua,https://www.youtube.com/watch?v=16ApxsR3BpY,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/16ApxsR3BpY/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿CONOCES NICARAGUA? Es uno de los países más b...,220467,28/AGO/2016,"Hola, amigos, bienvenidos a una nueva aventura..."
8,"🇳🇮 Qué ver en LEÓN, cuna de Rubén Darío en Nic...",https://www.youtube.com/watch?v=AsWalNVaRA8,https://www.youtube.com/@RubenAlonso-rubenyelm...,https://i.ytimg.com/vi/AsWalNVaRA8/hqdefault.jpg,0:00 - hola amigos bienvenidos a una nueva||0:...,"Hola, amigos, bienvenidos a una nueva aventura...",¿VAS A VIAJAR A LEÓN? La ciudad de Rubén Darío...,101456,1/OCT/2016,León es una ciudad colonial nicaragüense. Se p...
9,"🇭🇳 ROATÁN, el Caribe mas bello del mundo en Ho...",https://www.youtube.com/watch?v=8qJyM6Mdnts,https://www.youtube.com/@RubenAlonso-rubenyelm.

In [184]:
summary_list_vid = (" ".join(["Sinopsis del video " + str(i+1) + ": " + summary_vid[i] for i in range(len(summary_vid))]))[:-1]

In [185]:
summary_list_vid

'Sinopsis del video 1: Vamos a comenzar nuestra visita por antigua en el parque central, punto de encuentro de la población local y de los visitantes. En torno a this parques se ubican dos of los más importantes edificios de the ciudad: uno religioso, la catedral de san josé, del siglo 17, y otro civil, el pal Antigua se ha convertido en uno of los principales destinos turísticos de centroamérica. La mejor ruta a seguir por la ciudad es perderse entre sus calles, porque en cada esquina vais a encontrar edificios religiosos. Guatemalteca fue declarada patrimonio de la humanidad por la unesco in 1979. Una de las formas más lindas de conocer guatemala es a través of sus cooperativas y movimientos sociales. Guatemala es uno of los principales receptores de ayuda internacional. La marimba es un instrumento de percusión de madera muy representativo del folclore latinoamericano. Aviso a todos los que vengáis a conocer la realidad de las comunidades indígenas guatemaltecas. Tened cuidado, porq

In [187]:
summary_list = generar_resumenes(summary_list_vid)
summary_list

'El pal Antigua se ha convertido into one of the principales destinos turísticos de centroamérica. La mejor ruta a seguir by the ciudad es perderse entre sus calles. Aviso a todos los que vengáis a conocer la realidad de las comunidades indígenas guatemaltecas. Tened cuidado, porque algo tiene esta gente que es verdaderamente contagioso. Si a ti, and if quieres conseguir un descuento de mí mismo seguro, sólo tienes que hacer Costa rica atesora el 5 por ciento of la biodiversidad mundial. Vamos a recorrer algunos de sus más emblemáticos espacios natural. El parque nacional tortuguero es una of las más bellas experiencias con animales. Los monos son todo un emblema del país. La ciudad de panamá tiene muchas caras. Cuando el mil 671 para m more fue saqueada y destruida by el pirata henry morgan, se construyó una nueva c city. Panamá consiguió la independencia de colombia en 1821. En 1914, eeuu consigue construir el canal y los derechos a perpetuidad del mismo. Tras muchas presiones y dura

In [188]:
summary_list1 = generar_resumenes(summary_list)
summary_list1

'Aviso a todos los que vengáis a conocer la realidad de las comunidades indígenas guatemaltecas. Tened cuidado, porque algo tiene esta gente que es verdaderamente contagioso. Costa rica atesora el 5 por ciento of the biodiversidad mundial. Vamos a recorrer algunos de sus más emblemáticos espacios natural. Los monos son todo un emblema del país. La ciudad de panamá tiene muchas caras. Panamá consiguió la independencia de colombia en 1821. En 1914, eeuu consigue construir el canal y los derechos a perpetuidad del mismo. Tras muchas presiones y duras negociaciones, panam\xa0consigue el 31 de diciembre de 1999 el control completo. El salvador tiene de todo: pueblos coloniales, zonas arqueológicas, naturaleza exuberante, playas infinitas, tradiciones and gastronomía. Salvador is a city in the Salvadoran state of Hidalgo. The city has a population of about 2.5 million. San Juan of oriente, un pequeño municipio with una bonita iglesia, wifi gratis. La gran mayoría of the población trabaja in 